# Checking the Quality of Synthesized Images

To automatically evaluate the structural integrity of augmentations in the image while retaining resemblance to the original,
- We use **BRISQUE**, a reference-free metric
which quantifies the perceptual quality of an image, labeling images with a score under 70 as highly
salient.
- Similarly, we use **CLIP** similarity between original and augmented images to ensure the diffusion model performed substantial enough augmentations on the original.

> ***Currently, the thresholds are arbitarily defined. We can discuss later to set them.***

> Nvidia GPU is needed to run this notebook. If not, recommended to run it on Google Colab. [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1bjf_09KLvF4Ml2pA-jLAX2E-pAHIm7HO)

Reference: [Semi-Truths: A Large-Scale Dataset of AI-Augmented
Images for Evaluating Robustness of AI-Generated
Image detectors](https://arxiv.org/pdf/2411.07472)

## Environmental Setup

In [1]:
!pip install -q brisque

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 5.6 MB/s eta 0:00:00


In [2]:
import torch
import pdb
import tqdm
import csv
import os
import sys
import numpy as np
import pandas as pd
import torchvision.transforms as transforms
from tqdm import tqdm
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer, CLIPImageProcessor
from PIL import Image, ImageFile
from brisque import BRISQUE

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
torch_device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_tensor_type("torch.cuda.FloatTensor")

model_ID = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_ID)
model.to(torch_device)

tokenizer = CLIPTokenizer.from_pretrained(model_ID)
preprocess = CLIPImageProcessor.from_pretrained(model_ID)

## Define Helper Functions

In [4]:
def preprocess_image(image):
    """
    Preprocess image using transformers.ClipImageProcessor.
    """
    image = preprocess(image, return_tensors="pt")
    return image


def calculate_image_similarity(image1, image2):
    image1 = preprocess_image(image1)["pixel_values"].to(torch_device)
    image2 = preprocess_image(image2)["pixel_values"].to(torch_device)
    image_encoding1 = model.get_image_features(image1)
    image_encoding2 = model.get_image_features(image2)
    similarity = torch.nn.functional.cosine_similarity(
        image_encoding1, image_encoding2, dim=-1
    )
    return similarity.item()


def brisque_Score(img):
    ndarray = np.asarray(img)
    obj = BRISQUE(url=False)
    score = obj.score(img=ndarray)
    return score

## Define Main Functions and Process

In [ ]:
def process_images(original_img_dir, synthesized_img_dir, output_csv_file):
    """
    Given an original image directory and synthesized image directory (containing multiple races),
    iterate through all images and calculate quality scores.
    """

    data = []

    for race_dir in os.listdir(synthesized_img_dir):

        if race_dir not in ['Asian', 'Black', 'Caucasian', 'Indian', 'White']:
          continue

        for filename in tqdm(os.listdir(original_img_dir)):
            if filename.endswith(('.png', '.jpg', '.jpeg')):
                original_img_path = os.path.join(original_img_dir, filename)

                filename_parts = filename.split('_')
                filename_parts.insert(1, race_dir)
                synthesized_filename = '_'.join(filename_parts)
                synthesized_path = os.path.join(synthesized_img_dir, race_dir, synthesized_filename)

                try:
                    orig_img = Image.open(original_img_path)
                    synthesized_img = Image.open(synthesized_path)

                    img_similarity = round(calculate_image_similarity(orig_img, synthesized_img),2)
                    brisque_score = round(brisque_Score(synthesized_img),2)

                    data.append([synthesized_filename, img_similarity, brisque_score])
                except FileNotFoundError:
                    print(f"Warning: Synthesized image not found for {synthesized_filename}. Skipping.")
                except Exception as e:
                    print(f"Error processing {synthesized_filename}: {e}")
        print(f"Scores for images in {race_dir} calculated.")

    processed_df = pd.DataFrame(data, columns=['filename', 'img_similarity', 'brisque_score'])

    processed_df.to_csv(output_csv_file, index=False)
    print(f"Results saved to {output_csv_file}")

    return processed_df

# given a csv file output df that filters out images that don't pass the criteria
def filter_images(csv_file, metric_thresholds):
    similarity_threshold = metric_thresholds['similarity']
    brisque_threshold = metric_thresholds['brisque']
    df = pd.read_csv(csv_file)
    failed_df = df[
        (df['img_similarity'] < similarity_threshold[0]) |
        (df['img_similarity'] > similarity_threshold[1]) |
        (df['brisque_score'] < brisque_threshold[0]) |
        (df['brisque_score'] > brisque_threshold[1])
            ]
    return failed_df

In [ ]:
category = 'Azerbaijani_Festival'
original_img_dir = f'drive/MyDrive/Team VQA - Datasets/{category}/original_images/'
synthesized_img_dir = f'drive/MyDrive/Team VQA - Datasets/{category}/synthesized_images/'
output_csv_file = f'{category}.csv'

processed_df = process_images(original_img_dir, synthesized_img_dir, output_csv_file)

100%|██████████| 34/34 [01:18<00:00,  2.30s/it]


Scores for images in Black calculated.


100%|██████████| 34/34 [00:56<00:00,  1.67s/it]


Scores for images in Indian calculated.


100%|██████████| 34/34 [00:56<00:00,  1.67s/it]


Scores for images in Caucasian calculated.


100%|██████████| 34/34 [00:57<00:00,  1.68s/it]

Scores for images in Asian calculated.
Results saved to Azerbaijani_Festival.csv


In [ ]:
metric_thresholds = {
    "similarity": [0.65, 0.98],
    "brisque": [0, 80],
}
failed_df = filter_images(output_csv_file, metric_thresholds)
failed_df

### Testing

Calculate BRISQUE score of images in a single directory.

In [15]:
# original_img_dir = f'drive/MyDrive/Team VQA - Datasets/US_Festival/original_images/'
original_img_dir = f'original_images/'

data = []
for filename in os.listdir(original_img_dir):
  if filename.endswith(('.png', '.jpg', '.jpeg')):
      original_img_path = os.path.join(original_img_dir, filename)

      try:
          orig_img = Image.open(original_img_path)

          brisque_score = round(brisque_Score(orig_img),2)

          data.append([filename, brisque_score])
      except FileNotFoundError:
          print(f"Warning: Synthesized image not found for {filename}. Skipping.")
      except Exception as e:
          print(f"Error processing {filename}: {e}")
processed_df = pd.DataFrame(data, columns=['filename', 'brisque_score'])

processed_df

Error processing US_Festival_6.png: the input array must have size 3 along `channel_axis`, got (571, 1015)
